In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

from copy import deepcopy


#For random forest for time series:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

#For metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")

#From Functions folder
from LoadSeries import load_series, moving_average
from Merge import merge
from Battery import Battery
from DPModel import DPModel, DPModel2
from Logic import logic_bat, logic_DP, logic_actions
from Funcs_Logic_DP import (get_price, get_emissions, logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, 
                            logic_series_print, DP_stochastic, policy_rollout)

In [2]:
house = "h16"
prod = load_series("p",house,"prod_"+house,[24,48,168])
cons = load_series("c",house,"cons_"+house,[24,48,168])
cons

,cons_h16,cons_h16-24,cons_h16-48,cons_h16-168,Year,Month,Day,Weekday,Hour
2020-12-29 00:00:00,5.7,5.7,6.9,6.3,2020,12,29,1,0
2020-12-29 01:00:00,5.9,5.4,5.3,4.8,2020,12,29,1,1
2020-12-29 02:00:00,5.0,6.0,4.9,4.8,2020,12,29,1,2
2020-12-29 03:00:00,3.9,6.8,4.4,4.3,2020,12,29,1,3
2020-12-29 04:00:00,3.5,4.0,3.6,4.0,2020,12,29,1,4
...,...,...,...,...,...,...,...,...,...
2023-01-10 16:00:00,6.1,7.6,6.9,5.4,2023,1,10,1,16
2023-01-10 17:00:00,8.3,10.0,9.8,6.2,2023,1,10,1,17
2023-01-10 18:00:00,10.4,9.1,14.5,13.0,2023,1,10,1,18
2023-01-10 19:00:00,9.9,11.1,10.8,10.4,2023,1,10,1,19


In [3]:
merged = merge(house)
merged

,prod_h16,cons_h16,power_yield,SpotPriceDKK,CO2Emission
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000,60.0
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000,58.0
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001,65.0
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998,67.0
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001,76.0
...,...,...,...,...,...
2022-12-31 19:00:00,0.0,8.7,-8.7,498.329987,209.0
2022-12-31 20:00:00,0.0,6.9,-6.9,301.190002,224.0
2022-12-31 21:00:00,0.0,5.3,-5.3,110.730003,253.0
2022-12-31 22:00:00,0.0,5.6,-5.6,73.919998,272.0


In [4]:
N = 24*7
bat_capacity=13
stepsize=2
Start="2022-06-19 00:00:00"
End = pd.date_range(start=Start, periods=N+(24-stepsize), freq="h")[-1]

train_prod = 2*7*24
train_cons = 3000

BeforeProd = pd.date_range(end=Start, periods=train_prod+1, freq="h")[0]
BeforeCons = pd.date_range(end=Start, periods=train_cons+1, freq="h")[0]

print(f"Period: From {Start} to {End}")

Period: From 2022-06-19 00:00:00 to 2022-06-26 21:00:00


In [5]:
forecasterProd = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=8,max_features=3,n_estimators=100,n_jobs=4),
                 lags      = list(range(1,10+1,1))
                )
forecasterCons = ForecasterAutoreg(
                 regressor = RandomForestRegressor(random_state=123,max_depth=8,max_features=4,n_estimators=100,n_jobs=4),
                 lags      = list(range(1,10+1,1))
                )

battery_no = Battery(max_capacity=bat_capacity,max_charge=0)
battery    = Battery(max_capacity=bat_capacity,max_charge=7)
DPbat      = Battery(max_capacity=bat_capacity,max_charge=7)
DP2bat     = Battery(max_capacity=bat_capacity,max_charge=7)

series_no_battery  = pd.DataFrame(columns=merged.columns)
series_battery     = pd.DataFrame(columns=merged.columns)
series_battery_DP  = pd.DataFrame(columns=merged.columns)
series_battery_DP2 = pd.DataFrame(columns=merged.columns)

preds = pd.DataFrame() 
for i in range(N//stepsize):#stepsize has to be even
    _, predProd = backtesting_forecaster(forecaster = forecasterProd,
                                         y          = prod.loc[BeforeProd:End].iloc[i*stepsize:train_prod+i*stepsize+24]['prod_'+house],
                                         exog       = prod.loc[BeforeProd:End].iloc[i*stepsize:train_prod+i*stepsize+24].drop(['prod_'+house], axis=1),
                                         initial_train_size = train_prod,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )

    _, predCons = backtesting_forecaster(forecaster = forecasterCons,
                                         y          = cons.loc[BeforeCons:End].iloc[i*stepsize:train_cons+i*stepsize+24]['cons_'+house],
                                         exog       = cons.loc[BeforeCons:End].iloc[i*stepsize:train_cons+i*stepsize+24].drop(['cons_'+house], axis=1),
                                         initial_train_size = train_cons,
                                         steps      = 24,
                                         refit      = True,
                                         metric     = 'mean_squared_error',
                                         fixed_train_size   = True,
                                         verbose    = False
                                        )
    pred_i = pd.merge(predProd, predCons, how="outer", left_index=True,right_index=True)
    pred_i = pred_i.rename(columns={"pred_x":"prod_"+house,"pred_y":"cons_"+house})

    pred_i["prod_"+house]=np.array((pred_i["prod_"+house].to_numpy())*10,dtype=int)/10
    pred_i["cons_"+house]=np.array((pred_i["cons_"+house].to_numpy())*10,dtype=int)/10

    pred_i["power_yield"] = pred_i["prod_"+house] - pred_i["cons_"+house]
    
    values = pred_i["power_yield"].to_numpy()
    
    for j in range(len(values)):
        values[j] = f"{values[j]:.1f}"
    
    pred_i["power_yield"] = values
    
    
    pred_i["SpotPriceDKK"] = merged.loc[pred_i.index[0]:pred_i.index[-1]]["SpotPriceDKK"]
    pred_i["CO2Emission"] = merged.loc[pred_i.index[0]:pred_i.index[-1]]["CO2Emission"]
    
    merged_i = merged.loc[Start:End].iloc[i*stepsize:i*stepsize+24]
    print(f"i: {i}, From {merged_i.index[0]} to {merged_i.index[-1]}")
    
    temp_no_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery_no), logic_bat)
    temp_bat = pred_logic_rollout(merged_i, pred_i, deepcopy(battery), logic_bat)

    DP = DPModel(24, pred_i.index[0], pred_i.index[-1], house, pred_i, deepcopy(DPbat))
    _, pi = DP_stochastic(DP)
    _, _, actions = policy_rollout(DP,pi=lambda x, k: pi[k][x],x0=DPbat.get_current_capacity())
    
    DP2 = DPModel2(24, pred_i.index[0], pred_i.index[-1], house, pred_i, deepcopy(DP2bat))
    _, pi2 = DP_stochastic(DP2)
    _, _, actions2 = policy_rollout(DP2,pi=lambda x, k: pi2[k][x],x0=DP2bat.get_current_capacity())
    
    print()
    
    series_no_battery_i  = action_rollout(merged_i.iloc[:stepsize], battery_no, temp_no_bat[:stepsize])
    series_battery_i     = action_rollout(merged_i.iloc[:stepsize], battery, temp_bat[:stepsize])
    series_battery_DP_i  = action_rollout(merged_i.iloc[:stepsize], DPbat, actions)
    series_battery_DP2_i = action_rollout(merged_i.iloc[:stepsize], DP2bat, actions2)
    
    
    preds=pd.concat([preds,pred_i[:stepsize]])
    series_no_battery  = pd.concat([series_no_battery,series_no_battery_i])
    series_battery     = pd.concat([series_battery,series_battery_i])
    series_battery_DP  = pd.concat([series_battery_DP,series_battery_DP_i])
    series_battery_DP2 = pd.concat([series_battery_DP2,series_battery_DP2_i])

series_no_battery["cost_cummulative"] = series_no_battery["cost"].cumsum(axis=0)
series_battery["cost_cummulative"] = series_battery["cost"].cumsum(axis=0) 
series_battery_DP["cost_cummulative"] = series_battery_DP["cost"].cumsum(axis=0)
series_battery_DP2["cost_cummulative"] = series_battery_DP2["cost"].cumsum(axis=0)
series_no_battery["emission_cummulative"] = series_no_battery["emission"].cumsum(axis=0)
series_battery["emission_cummulative"] = series_battery["emission"].cumsum(axis=0) 
series_battery_DP["emission_cummulative"] = series_battery_DP["emission"].cumsum(axis=0) 
series_battery_DP2["emission_cummulative"] = series_battery_DP2["emission"].cumsum(axis=0) 
preds

i: 0, From 2022-06-19 00:00:00 to 2022-06-19 23:00:00
0  
i: 1, From 2022-06-19 02:00:00 to 2022-06-20 01:00:00
0  
i: 2, From 2022-06-19 04:00:00 to 2022-06-20 03:00:00
0  
i: 3, From 2022-06-19 06:00:00 to 2022-06-20 05:00:00
0  
i: 4, From 2022-06-19 08:00:00 to 2022-06-20 07:00:00
0  
i: 5, From 2022-06-19 10:00:00 to 2022-06-20 09:00:00
0  
i: 6, From 2022-06-19 12:00:00 to 2022-06-20 11:00:00
0  
i: 7, From 2022-06-19 14:00:00 to 2022-06-20 13:00:00
0  
i: 8, From 2022-06-19 16:00:00 to 2022-06-20 15:00:00
0  
i: 9, From 2022-06-19 18:00:00 to 2022-06-20 17:00:00
0  
i: 10, From 2022-06-19 20:00:00 to 2022-06-20 19:00:00
0  
i: 11, From 2022-06-19 22:00:00 to 2022-06-20 21:00:00
0  
i: 12, From 2022-06-20 00:00:00 to 2022-06-20 23:00:00
0  
i: 13, From 2022-06-20 02:00:00 to 2022-06-21 01:00:00
0  
i: 14, From 2022-06-20 04:00:00 to 2022-06-21 03:00:00
0  
i: 15, From 2022-06-20 06:00:00 to 2022-06-21 05:00:00
0  
i: 16, From 2022-06-20 08:00:00 to 2022-06-21 07:00:00
0  
i: 17, 

,prod_h16,cons_h16,power_yield,SpotPriceDKK,CO2Emission
2022-06-19 00:00:00,0.0,6.3,-6.3,1540.390015,137.0
2022-06-19 01:00:00,0.0,5.7,-5.7,1413.410034,102.0
2022-06-19 02:00:00,0.0,4.5,-4.5,1231.300049,146.0
2022-06-19 03:00:00,0.0,3.9,-3.9,1039.969971,155.0
2022-06-19 04:00:00,0.0,3.6,-3.6,897.140015,155.0
...,...,...,...,...,...
2022-06-25 19:00:00,10.8,10.5,0.3,2336.860107,104.0
2022-06-25 20:00:00,5.1,9.1,-4.0,2434.100098,103.0
2022-06-25 21:00:00,1.0,8.3,-7.3,2400.699951,99.0
2022-06-25 22:00:00,0.3,7.2,-6.9,2410.669922,86.0


In [6]:
print_price_summary(series_no_battery)
logic_series_print(series_no_battery)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  802.0  DKK
Total emissions for period:  64.0  kg
Average cost per year is: 42077.0 DKK
Average emissions per year is: 3354.0 kg
Number of kwh purchased in the period: 456.3
Average number of kwh purchased per year: 23951.651497005987
Average number of kwh sold per year: 34334.3748502994
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0

   81:   2.5277,  0.1590, -1.9000, -1.9000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  4.8026,342.8651,  0.3021, 34.8944
   82:   2.2428,  0.1340, -2.3000, -2.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1584,348.0235,  0.3082, 35.2026
   83:   2.1754,  0.1510,  1.2000,  1.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.2611,347.7625,  0.0000, 35.2026
   84:   2.0003,  0.1370,  0.3000,  0.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.0600,347.7025,  0.0000, 35.2026
   85:   1.8575,  0.1440,  3.8000,  3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.7059,346.9966,  0.0000, 35.2026
   86:   1.8185,  0.1270, -1.7000, -1.7000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.0915,350.0881,  0.2159, 35.4185
   87:   1.9201,  0.1060,  1.3000,  1.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.2496,349.8385,  0.0000, 35.4185
   88:   2.1134,  0.1210,  1.3000,  1.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.2747,349.5637,  0.0000, 35.4185
   89:  

In [7]:
print_price_summary(series_battery)
logic_series_print(series_battery)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  688.0  DKK
Total emissions for period:  63.0  kg
Average cost per year is: 36131.0 DKK
Average emissions per year is: 3283.0 kg
Number of kwh purchased in the period: 420.8
Average number of kwh purchased per year: 22088.220359281437
Average number of kwh sold per year: 29862.140119760475
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:  

  118:   2.1749,  0.0910, -6.5000, -2.1000,  0.0000, -4.4000,  4.7000,  4.4000,  0.0000,  4.5673,494.9888,  0.2184, 49.4006
  119:   1.5650,  0.1000, -7.6000, -7.6000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 11.8938,506.8826,  0.7600, 50.1606
  120:   1.6726,  0.0680, -5.6000, -5.6000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  9.3663,516.2489,  0.3808, 50.5414
  121:   1.5747,  0.0690, -5.4000, -5.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.5035,524.7524,  0.3726, 50.9140
  122:   1.5191,  0.0600, -4.5000, -4.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  6.8359,531.5883,  0.2700, 51.1840
  123:   1.4320,  0.0620, -3.7000, -3.7000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.2983,536.8867,  0.2294, 51.4134
  124:   1.4010,  0.0760, -3.4000, -3.4000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  4.7635,541.6502,  0.2584, 51.6718
  125:   1.4062,  0.0810, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  4.9218,546.5720,  0.2835, 51.9553
  126:  

In [8]:
print_price_summary(series_battery_DP)
logic_series_print(series_battery_DP)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  668.0  DKK
Total emissions for period:  62.0  kg
Average cost per year is: 35083.0 DKK
Average emissions per year is: 3264.0 kg
Number of kwh purchased in the period: 432.1
Average number of kwh purchased per year: 22681.36886227545
Average number of kwh sold per year: 31158.668263473053
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   

   74:   2.0003,  0.1730, -4.0000, -4.0000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.0013,240.9478,  0.6920, 30.8391
   75:   1.8486,  0.1680, -4.2000, -8.2000,  4.0000,  4.0000,  0.0000,  0.0000,  4.0000, 15.1584,256.1062,  1.3776, 32.2167
   76:   1.8971,  0.1350, -3.3000, -9.5000,  6.2000,  6.2000,  4.0000,  3.8000, 10.0000, 18.0223,274.1284,  1.3095, 33.5262
   77:   2.1148,  0.1730, -3.5000,  0.0000,  0.0000, -3.5000, 10.0000,  9.5000,  6.0000, -0.0000,274.1284,  0.0865, 33.6127
   78:   2.5924,  0.1950, -3.2000, -0.4000,  0.0000, -2.8000,  6.0000,  5.7000,  2.9000,  1.0370,275.1654,  0.1365, 33.7492
   79:   3.1984,  0.1800, -1.7000,  1.0000,  0.0000, -2.7000,  2.9000,  2.7000,  0.0000, -0.3198,274.8456,  0.0000, 33.7492
   80:   2.8854,  0.1700, -2.2000, -2.2000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  6.3479,281.1935,  0.3740, 34.1232
   81:   2.5277,  0.1590, -1.9000, -1.9000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  4.8026,285.9961,  0.3021, 34.4253
   82:  

In [9]:
print_price_summary(series_battery_DP2)
logic_series_print(series_battery_DP2)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  705.0  DKK
Total emissions for period:  58.0  kg
Average cost per year is: 36985.0 DKK
Average emissions per year is: 3028.0 kg
Number of kwh purchased in the period: 423.8
Average number of kwh purchased per year: 22245.693413173653
Average number of kwh sold per year: 31358.13413173653
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000,-12.3000,  7.0000,  7.0000,  0.0000,  0.0000,  7.0000, 17.3849, 25.8571,  1.2546,  2.0081
    2:   1.2313,  0.1460, -4.2000,  0.3000,  0.0000, -4.5000,  7.0000,  6.6000,  2.0000, -0.0369, 25.8201,  0.0146,  2.0227
    3:   1.0400,  0.1550, -3.8000, -1.9000,  0.0000, -1.9000,  2.0000,  1.9000,  0.0000,  1.9759, 27.7961,  0.3100,  2.3327
    4:   

   84:   2.0003,  0.1370,  0.3000,  0.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.0600,297.4914,  0.0000, 31.4315
   85:   1.8575,  0.1440,  3.8000,  3.8000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.7059,296.7855,  0.0000, 31.4315
   86:   1.8185,  0.1270, -1.7000, -1.7000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000,  3.0915,299.8770,  0.2159, 31.6474
   87:   1.9201,  0.1060,  1.3000,  1.3000,  0.0000, -0.0000,  0.0000,  0.0000,  0.0000, -0.2496,299.6274,  0.0000, 31.6474
   88:   2.1134,  0.1210,  1.3000, -2.1000,  2.1000,  3.4000,  0.0000,  0.0000,  3.4000,  4.4382,304.0656,  0.2541, 31.9015
   89:   2.4658,  0.1360, -1.0000, -3.8000,  2.8000,  2.8000,  3.4000,  3.2000,  6.0000,  9.3700,313.4356,  0.5440, 32.4455
   90:   2.9634,  0.1440, -5.5000, -3.8000,  0.0000, -1.7000,  6.0000,  5.7000,  4.0000, 11.2608,324.6964,  0.5904, 33.0359
   91:   3.6528,  0.1550, -8.1000, -4.3000,  0.0000, -3.8000,  4.0000,  3.8000,  0.0000, 15.7071,340.4036,  0.6975, 33.7334
   92:  